### Deliverable 1: Preprocessing the Data for a Neural Network

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)
app_type_cnts=application_df["APPLICATION_TYPE"].value_counts(dropna=False)
# Determine which values to replace if counts are less than ...?
replace_application=list(app_type_cnts[app_type_cnts<500].index)
# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
# Look at CLASSIFICATION value counts for binning
class_type_cnts=application_df["CLASSIFICATION"].value_counts(dropna=False)
# Determine which values to replace if counts are less than ..?
replace_class=list(class_type_cnts[class_type_cnts<1883].index)
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
# Generate our categorical variable lists
application_cat =application_df.dtypes[application_df.dtypes =="object"].index.tolist()
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df =pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
# Merge one-hot encoded features and drop the originals
application_df =application_df.merge(encode_df,left_index=True,right_index=True)
application_df =application_df.drop(application_cat,1)
# Split our preprocessed data into our features and target arrays
y =application_df["IS_SUCCESSFUL"].values
X =application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test =train_test_split(X,y,random_state=78)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# number of input features = 43 columns in X, suggests 86-129 nodes in hidden layers
number_input_features =len(X_train[0])
hidden_nodes_layer1 =80
hidden_nodes_layer2 = 30
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation="relu"))
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2,activation="relu"))
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

In [ ]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint
# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path ="checkpoints/weights.{epoch:02d}.hdf5"
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every epoch
cp_callback =ModelCheckpoint(filepath=checkpoint_path,verbose=1,save_weights_only=True,save_freq=5)

In [ ]:
# Train the model
fit_model =nn.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Create a DataFrame containing training history
history_df =pd.DataFrame(fit_model.history,index=range(1,len(fit_model.history["loss"])+1))
# Plot the loss
history_df.plot(y="loss")


In [ ]:
# Plot the accuracy
history_df.plot(y="accuracy")

In [ ]:
# Export our model to HDF5 file
nn.save("trained_charity_model.h5")